In [1]:
import numpy as np
from keras import Sequential
from keras.layers import Dense, Dropout

Pretraining the trans function for a garch(2,1) model

In [45]:
# create dataset
s1 = 50
s2 = 50
s3 = 23
const = np.random.uniform(low=0, high=0.01, size=s1 * s2 * s3)
q1s = np.tile(np.linspace(0.03, 0.28, s1), s2 * s3) + np.random.normal(loc=0, scale=0.002, size=s1 * s2 * s3)
q2s = np.tile(np.linspace(0, 0.25, s2), s1 * s3) + np.random.normal(loc=0, scale=0.002, size=s1 * s2 * s3)
p1s = np.tile(np.linspace(0.5, 1, s3), s1 * s2) + np.random.normal(loc=0, scale=0.002, size=s1 * s2 * s3)
q1s = np.random.uniform(low=0.03, high=0.28, size=s1 * s2 * s3)
q2s = np.random.uniform(low=0, high=0.25, size=s1 * s2 * s3)
p1s = np.random.uniform(low=0.5, high=1, size=s1 * s2 * s3)
r1s = np.random.uniform(low=-0.04, high=0.04, size=s1 * s2 * s3)
r2s = np.random.uniform(low=-0.04, high=0.04, size=s1 * s2 * s3)
cv1s = np.random.uniform(low=0, high=0.04, size=s1 * s2 * s3)
xs = np.zeros((s1 * s2 * s3, 7))
ys = np.zeros((s1 * s2 * s3, 1))
i = 0
for q1_i in range(0, s1):
    for q2_i in range(0, s2):
        for p1_i in range(0, s3):
            xs[i][0] = const[i]
            xs[i][1] = q1s[i]
            xs[i][2] = q2s[i]
            xs[i][3] = p1s[i]
            xs[i][4] = cv1s[i]
            xs[i][5] = r2s[i]
            xs[i][6] = r1s[i]
            ys[i][0] = const[i] + q1s[i] * (r1s[i] ** 2) + q2s[i] * (r2s[i] ** 2) + p1s[i] * (cv1s[i] ** 2)
            i += 1

print(xs.mean(axis=0))
print(xs.std(axis=0))
print(ys.mean(axis=0))
print(ys.std(axis=0))

xs = (xs - xs.mean(axis=0)) / xs.std(axis=0)
ys = (ys - ys.mean(axis=0)) / ys.std(axis=0)

[ 4.97861683e-03  1.54884455e-01  1.24940350e-01  7.50835131e-01
  1.99980894e-02  7.63748711e-05 -2.50800646e-05]
[0.0028861  0.07228504 0.07229742 0.1444811  0.01155304 0.02310782
 0.02304983]
[0.00552819]
[0.00291117]


In [52]:
model = Sequential()
l1_size = 400
l2_size = 400
l3_size = 400
l4_size = 200
# 4 parameters, 1 prev vols, 2 prev innovations = 7 inputs
model.add(Dense(l1_size, input_shape=(7,), activation='tanh'))
model.add(Dropout(.2, input_shape=(l1_size,)))
model.add(Dense(l2_size, input_shape=(l1_size,), activation='tanh'))
model.add(Dropout(.2, input_shape=(l2_size,)))
model.add(Dense(l3_size, input_shape=(l2_size,), activation='tanh'))
model.add(Dropout(.2, input_shape=(l3_size,)))
model.add(Dense(l4_size, input_shape=(l3_size,), activation='tanh'))
model.add(Dropout(.2, input_shape=(l4_size,)))
model.add(Dense(1, input_shape=(l4_size,), activation='linear'))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

model.fit(xs, ys, epochs=20, batch_size=25)

Epoch 1/20
2300/2300 [==============================] - 11s 4ms/step - loss: 0.0284 - accuracy: 0.0000e+00
Epoch 2/20
2300/2300 [==============================] - 10s 5ms/step - loss: 0.0145 - accuracy: 0.0000e+00
Epoch 3/20
2300/2300 [==============================] - 11s 5ms/step - loss: 0.0145 - accuracy: 0.0000e+00
Epoch 4/20
2300/2300 [==============================] - 10s 4ms/step - loss: 0.0148 - accuracy: 0.0000e+00
Epoch 5/20
2300/2300 [==============================] - 10s 4ms/step - loss: 0.0154 - accuracy: 0.0000e+00
Epoch 6/20
2300/2300 [==============================] - 10s 4ms/step - loss: 0.0147 - accuracy: 0.0000e+00
Epoch 7/20
2300/2300 [==============================] - 10s 5ms/step - loss: 0.0150 - accuracy: 0.0000e+00
Epoch 8/20
2300/2300 [==============================] - 10s 5ms/step - loss: 0.0151 - accuracy: 0.0000e+00
Epoch 9/20
2300/2300 [==============================] - 10s 5ms/step - loss: 0.0154 - accuracy: 0.0000e+00
Epoch 10/20
2300/2300 [==============

In [53]:
dropout_layers = 0
for layer in range(0, 5):
    np.save(f"./pretrained_weights/garch_trans_layer_{layer}_weights", model.layers[layer + dropout_layers].get_weights()[0])
    np.save(f"./pretrained_weights/garch_trans_layer_{layer}_biases", model.layers[layer + dropout_layers].get_weights()[1])
    dropout_layers += 1

In [54]:
import matplotlib.pyplot as plt


ys_pred = model.predict(xs)
ys_pred = ys_pred * 0.00291117 + 0.00552819
ysun = ys * 0.00291117 + 0.00552819
print(((ys_pred - ysun)**2).mean())



1797/1797 [==============================] - 2s 1ms/step
5.378273904164622e-08
